<a href="https://colab.research.google.com/github/alinapradhan/Image-Generator-using-Stable-Diffusion/blob/main/ImageGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies (run this once per environment)
!pip install diffusers transformers accelerate torch safetensors
!pip install Pillow

# Optional: Log in to Hugging Face if the model requires authentication
from huggingface_hub import login

# Uncomment and enter your Hugging Face token here if needed:
# login("YOUR_HUGGINGFACE_TOKEN")


In [ ]:
# Import required libraries
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
from IPython.display import display

# Load the Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
)
pipe = pipe.to("cuda")  # use GPU if available


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [3]:
# Define your text prompt
prompt = "A futuristic city skyline at sunset, in the style of a digital painting"

# Generate the image
image = pipe(prompt).images[0]

# Display the image
display(image)

# Optionally, save the image
image.save("generated_image.png")


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from IPython.display import display
import ipywidgets as widgets

prompt_box = widgets.Text(
    value='A magical forest with glowing mushrooms',
    placeholder='Type your image description...',
    description='Prompt:',
    disabled=False
)

button = widgets.Button(description="Generate Image")

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        print(f"Generating image for: '{prompt_box.value}'...")
        image = pipe(prompt_box.value).images[0]
        display(image)

button.on_click(on_button_clicked)
display(prompt_box, button, output)
